In [11]:
import pandas as pd
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Libraries imported")
import numpy as np

Libraries imported


In [12]:
df=pd.read_excel(r'C:\Users\PALA\Desktop\week3\toronto.xlsx')


In [13]:
df.drop(df[df.Borough=="Not assigned"].index,inplace=True) # dropping "not assigned " values for  borough column
df.Borough.unique()

array(['York', 'West Toronto', 'Scarborough', 'North York', 'Mississauga',
       'Etobicoke', 'East York', 'East Toronto', 'Downtown Toronto',
       'Central Toronto'], dtype=object)

In [14]:
coordinates=pd.read_csv(r"C:\Users\PALA\Desktop\week3\Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
coordinates.set_index("Postal Code",inplace=True) # Postal code common index for both coordinates table and neigbourhood table
coordinates.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [16]:
df.set_index("Postal Code",inplace=True) # Postal code common index for both coordinates table and neigbourhood table 
df.head()

,Borough,Neighbourhood
Postal Code,,
M6C,York,Humewood-Cedarvale
M6E,York,Caledonia-Fairbanks
M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
M6N,York,"Runnymede, The Junction North"
M9N,York,Weston


In [17]:
df_latlon=pd.concat([df,coordinates],axis=1) # after we set postal code as index, now we re ready to concat process
df_latlon.head() # this is our new dataframe 

C:\Users\PALA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Borough,Neighbourhood,Latitude,Longitude
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
df_latlon.reset_index(inplace=True) # I reset index to numeric  values 
df_latlon.head()

,index,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
df_latlon.rename(columns={"index":"Postal Code"},inplace=True) # I rename postal code column from "index" to "Postal Code"
df_latlon.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [20]:
selection=df_latlon[df_latlon.Borough.str.contains("Toronto")] # Clustering process only based on Boroughs that their name contains word "Toronto"
selection.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
pd.value_counts(selection.Borough) # we have 4 boroughs that their names contains "toronto",and 39 neighbourhoods


Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [22]:
latitude=43.6532 #Toronto map coordinates
longitude=-79.3832 #Toronto map coordinates
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10) #Let's create Toronto map 
for lat,lng,borough,neighbourhood in zip(selection["Latitude"],selection["Longitude"],selection["Borough"],selection["Neighbourhood"]):
    label="{},{}".format(borough,neighbourhood)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color="blue",
    fill=True,fill_color='#3186cc',
    fill_opacity=0.7,parse_html=False).add_to(map_toronto)

map_toronto # this map shows neighbourhoods of 4 borough before clustering 
